### Posibilidad de identificación de un sujeto despues de la anonimización de los datos

Actualmente la anonimización realizada por el departamento de gestión académica que nos proporcionó los datos consiste en cambiar el NIUB de los alumnos por un identificador único que no tiene ninguna relación alfa-numérica con el \textit{NIUB}. Es decir, sabiendo solo los identificadores de los alumnos es imposible saber de quién se trata realmente debido a que  no se conoce la manera ni el orden de asignar los nuevos identificadores.

Debido a que los datos no se tienen que publicar actualmente, no se ha aplicado ninguna técnica de anonimización potente. Por este motivo queremos saber si, disponiendo de varios datos sobre un sujeto en particular, se podría llegar a identificarlo.

Por ejemplo, se exponen los siguientes casos hipotéticos que podrían llevar a la identificación de un sujeto

In [1]:
import numpy as np
import pandas as pd
import datetime

#####Función que permite selecionar un subconjunto que cumpla con una determinada condición de un dataFrame

In [2]:
def select_rows(df, group, col):
    #Creamos un nueva DataFrame
    out = pd.DataFrame()
    tmp = df.copy()
    # Recorremos los grupos
    for s in group:
        # Añadimos al DataFrame el nuevo elemento
        out = pd.concat([out, tmp[tmp[col] == s]])
    out = out.reset_index(drop=True)
    return out

##### Leemos los datos cargando las tablas necesarias

In [3]:
registers = pd.read_csv("datos3/registers.csv", na_values=np.nan)
qualifications = pd.read_csv("datos3/qualifications.csv", na_values=np.nan)

##### Unimos la tabla que representa los registros de los alumnos con la tabla que representa las calificaciones. De este modo tendremos por cada linea, una calificación, junto con toda la información del alumno.

In [4]:
datos = pd.merge(registers, qualifications)

#####A continuación intentaremos a partir de unos casos ipoteticos la identificación de una o varias personas
###Ejemplo 1. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Año de nacimiento: 1986
+ Sexo: hombre
+ Población del centro de secundaria: Castelldefels (comarca Baix Llobregat)

A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Primero filtramos los registros por el año de nacimeinto de las personas: 1986

In [5]:
datos1 = datos[datos['naixement'] == 1986].reset_index(drop=True)
print "Numero de alumnos nacido el 1986: " + str(len(datos1.id_alumne.unique()))

Numero de alumnos nacido el 1986: 22


#####Podremos ver que la lista de alumnos se ha reducido a 22.

A continuación filtraremos los registros segun el sexo de tal manera que nos quedemos solo con los varones.


In [6]:
datos1 = datos1[datos1['sexe'] == 'H'].reset_index(drop=True)
print "Numero de alumnos varones que han nacido el 1986: " + str(len(datos.id_alumne.unique()))

Numero de alumnos varones que han nacido el 1986: 803


#####Podremos ver que la lista de alumnos se ha reducido a 21.

A continuación filtraremos los registros segun la comarca donde han cursado la secundario, en este caso Baix Llobregat

In [7]:
datos1 = datos1[datos1['lloc_secundaria'] == 'Baix Llobregat'].reset_index(drop=True)
print "Numero de alumnos varones que han nacido el 1986 y que han cursado la secundaria en la comarca de Baix Llobregat: " + str(len(datos1.id_alumne.unique()))

Numero de alumnos varones que han nacido el 1986 y que han cursado la secundaria en la comarca de Baix Llobregat: 1


##### Como podremos ver con estos tres datos que sabiamos sobre una persona pudimos identificarla dentro de la base de datos de este modo pudiendo ver toda su información.

#### Una posible solución al problema de este caso sería añadir un pequeño ruido a los datos que representan los años o subir el nivel que de la area administrativa a una provincia en vez de una comarca.

###Ejemplo 2. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Nota de las pruebas PAU: 6.915


A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Filtramos los registros por la nota de las Pruebas PAU: 6.915

In [8]:
datos2 = datos[datos['nota_acces'] == 6.915].reset_index(drop=True)
print "Numero de alumnos con la nota de las pruebas PAU igual a 6.915: " + str(len(datos2.id_alumne.unique()))

Numero de alumnos con la nota de las pruebas PAU igual a 6.915: 1


##### Como podremos ver solo con saber la nota de las pruebas PAU de una persona, pudimos identificarla dentro de la base de datos

#### Podríamos solucionar este problema añadiendo un ruido a las notas de acceso o quitar decimales

###Ejemplo 3. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Notas de las asignaturas cursadas: 5.8, 8.9, 5.5, 6.2, 7.8


A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Filtramos los registros por la lista de notas mas arriba indicados.

In [9]:
notas = [5.8, 8.9, 5.5, 6.2, 7.8]

# Nos quedamos con los registros que tiene estas notas
uno = datos[datos['nota_primera_conv'] == 5.8].reset_index(drop=True)
dos = datos[datos['nota_primera_conv'] == 8.9].reset_index(drop=True)
tres = datos[datos['nota_primera_conv'] == 5.5].reset_index(drop=True)
cuatro = datos[datos['nota_primera_conv'] == 6.2].reset_index(drop=True)
cinco = datos[datos['nota_primera_conv'] == 7.8].reset_index(drop=True)

# Concatenamos los DataFrames
datos3 = pd.concat([uno, dos, tres, cuatro, cinco])
# Selecionamos los identificadors de los alumnos
ids = datos3.id_alumne.unique()
# Agrupamos los datos en funcíon de los identificadores de los alumnos
gg = datos3.groupby('id_alumne')

id_alumnes = []
# Recorremos los identificadores de los alumnos
for id_alumne in ids:
    # Obtenemos las notas del alumno actual
    alumne = gg.get_group(id_alumne)['nota_primera_conv'].values
    # Si ele alumnos acual tiene al menos el mismo numero de notas que las definicad anteriormente, continuamos con la verificación
    if (len(alumne) >= len(notas)):
        k = 0
        # Recorremos las notas definidas anteriormente
        for nota in notas:
            if nota in alumne:
                k+=1
        # Si el alumnos dispone de todas las notas, lo guardamos
        if k == len (notas):
            id_alumnes.append(gg.get_group(id_alumne)['id_alumne'].unique())
            

print "Numero de alumnos encontrados con las siguientes notas " + str(notas) + ": " + str(len(id_alumnes))

Numero de alumnos encontrados con las siguientes notas [5.8, 8.9, 5.5, 6.2, 7.8]: 1


#### Podríamos solucionar este problema añadiendo un ruido a las notas de acceso o quitar decimales

###Ejemplo 4. 

Supongamos que disponemos de la sigueinte información acerca de una persona:

+ Nota de la asignatura cursada de Algebra (364291): 8.5
+ Nota de la asignatura cursada de : Electronica (364305): 7.7


A continuación se mostrara que con saber esta información podríamos identificar a esta persona dentro de la base de datos y acceder a la información relacionada.

Filtramos los registros por la lista de notas mas arriba indicados.

In [10]:
asignaturas = [364291, 364305]
# Nos quedamos solo con las calificaciones de las dos asignaturas
datos4_A = datos[datos['id_assig'] == 364291].reset_index(drop=True)
datos4_B = datos[datos['id_assig'] == 364305].reset_index(drop=True)
# Nos quedamos solo con las calificaciones que son iguales a las dos ya establecidas
datos4_A = datos4_A[datos4_A['nota_primera_conv'] == 8.5]
datos4_B = datos4_B[datos4_B['nota_primera_conv'] == 7.7]

In [11]:
datos4 = pd.concat([datos4_A, datos4_B])
# Obtenemos los identificadores de los usuarios
ids = datos4.id_alumne.unique()
# Agrupamos las calificaciones por los identificadores de los usuarios
datos4 = datos4.groupby('id_alumne')

id_alumnes = []
# Recorremos la lista de los usuarios
for id_alumne in ids:
    # Obtenemos las notas del alumno actual
    gr = datos4.get_group(id_alumne)['id_assig'].values
    # Si ele alumnos acual tiene al menos el mismo numero de notas que las definicad anteriormente, continuamos con la verificación
    if (len(gr) >= len(asignaturas)):
        k = 0
        # Recorremos las notas definidas anteriormente
        for asig in asignaturas:
            if asig in gr:
                k+=1
        # Si el alumnos dispone de todas las notas, lo guardamos
        if k == len (asignaturas):
            id_alumnes.append(datos4.get_group(id_alumne)['id_alumne'].unique())
        
print "Numero de alumnos encontrados: " + str(len(id_alumnes))

Numero de alumnos encontrados: 1


#### Podríamos solucionar este problema añadiendo un ruido a las notas de acceso o quitar decimales

##### A continuación podemos ver las columnas disponibles en la tabla de los registros

In [12]:
print registers.columns

Index([u'id_alumne', u'sexe', u'naixement', u'nacionalitat', u'simultaneitat', u'becat', u'priv_pub_cfgs', u'tipus_lloc_cfgs', u'lloc_cfgs', u'priv_pub_secundaria', u'tipus_lloc_secundaria', u'lloc_secundaria', u'desc_via_acces', u'id_via_acces', u'any_pau', u'universitat_procedencia', u'universitat_procedencia_desc', u'sistema_educatiu_estranger', u'pais_sistema_estranger', u'nota_acces', u'desc_enseny', u'id_enseny', u'any_primera_matricula'], dtype='object')


##### Selecionamos aquellas columnas que creemos que podrían hacer posible la desanonimización de los alumnos

In [13]:
reg = registers[[u'sexe', u'naixement', u'nacionalitat', u'simultaneitat', u'becat', u'priv_pub_cfgs', u'tipus_lloc_cfgs', u'lloc_cfgs', u'priv_pub_secundaria', u'tipus_lloc_secundaria', u'lloc_secundaria', u'id_via_acces', u'any_pau', u'universitat_procedencia', u'sistema_educatiu_estranger', u'pais_sistema_estranger', u'nota_acces', u'id_enseny', u'any_primera_matricula']]

##### Obtenemos los valores unicos de todas las columnas

In [14]:
d = {}
# Recorremos todas las columnas
for a in reg.columns:
    # Obtenemos los valores unicos de la columna actual
    lista = reg.get(a).unique()
    # Eliminamos los valores nan
    cleanedList = [x for x in lista if str(x) != 'nan']
    d[a] = cleanedList

##### A continuación verificaremos cuantos alumnos se pueden identificar con la premisa de que se pueden conocer DOS de dos columnas distintas

In [15]:
# Guardaremos los resultados en un diccionario
dic = {}
coin = 0
no_coin = 0
# Obtenemos los nombres de las columnas disponiles
ke = d.keys()
# Recorremos las columnas
while (len(ke) > 0):
    aa = ke.pop()
    # Recorremos los valores de la columna actual
    for a in d.get(aa):
        ke2 = list(ke)
        # Recorremos las columnas
        while (len(ke2) > 0):
            bb = ke2.pop()
            # Recorremos los valores de la columna actual
            for b in d.get(bb):
                regi = reg
                # Aplicamos los filtros con los dos datos que conocemos
                regi = regi[regi[aa] == a]
                regi = regi[regi[bb] == b]
                par = dic.get((aa, bb))
                # Guardamos el numero de indentificaciones
                if (par == None):
                    dic[(aa, bb)] = (0,0)
                x, y = dic.get((aa, bb))
                if (0 < regi.shape[0] < 5):
                    coin += 1
                    x += 1
                    y += 1
                else:
                    no_coin += 1
                    y += 1
                dic[(aa, bb)] = (x,y)

##### Calculamos los porcentajes de identificación por cada par posible de columnas

In [16]:
new_d = {}
for key, value in dic.iteritems():
    a, b = value
    if (a == 0 or b == 0):
        new_d[key] = 0
    else:
        new_d[key] = a * 1.0 / b

##### Ordenamos los procentajes descendentemente y los mostramos

In [17]:
import operator
sorted_x = sorted(new_d.items(), key=operator.itemgetter(1), reverse = True)

for i in sorted_x:
    print i

(('lloc_cfgs', 'tipus_lloc_cfgs'), 1.0)
(('priv_pub_cfgs', 'lloc_cfgs'), 0.75)
(('id_enseny', 'lloc_cfgs'), 0.75)
(('lloc_secundaria', 'sexe'), 0.6296296296296297)
(('lloc_secundaria', 'becat'), 0.6296296296296297)
(('lloc_cfgs', 'sexe'), 0.625)
(('lloc_cfgs', 'simultaneitat'), 0.625)
(('pais_sistema_estranger', 'sexe'), 0.625)
(('lloc_cfgs', 'becat'), 0.625)
(('id_enseny', 'pais_sistema_estranger'), 0.625)
(('id_enseny', 'nacionalitat'), 0.6041666666666666)
(('pais_sistema_estranger', 'simultaneitat'), 0.5625)
(('id_enseny', 'nota_acces'), 0.5549668874172186)
(('nota_acces', 'becat'), 0.5516556291390728)
(('nota_acces', 'sexe'), 0.5410596026490067)
(('nota_acces', 'simultaneitat'), 0.5052980132450331)
(('tipus_lloc_cfgs', 'simultaneitat'), 0.5)
(('id_enseny', 'tipus_lloc_cfgs'), 0.5)
(('priv_pub_cfgs', 'sexe'), 0.5)
(('tipus_lloc_secundaria', 'simultaneitat'), 0.5)
(('sistema_educatiu_estranger', 'priv_pub_secundaria'), 0.5)
(('id_enseny', 'priv_pub_cfgs'), 0.5)
(('priv_pub_cfgs', 'be

##### A continuación verificaremos cuantos alumnos se pueden identificar con la premisa de que se puede conocer UN dato de cualquier columna

In [18]:
# Guardaremos los resultados en un diccionario
dic = {}
coin = 0
no_coin = 0
# Obtenemos los nombres de las columnas disponibles
ke = d.keys()
# Recorremos las columnas
while (len(ke) > 0):
    aa = ke.pop()
    # Recorremos los valores de la columna actual
    for a in d.get(aa):
        regi = reg
        # Aplicamos los filtros con el dato que se conoce
        regi = regi[regi[aa] == a]
        par = dic.get(aa)
        # Guardamos el numero de indentificaciones
        if (par == None):
            dic[aa] = (0,0)
        x, y = dic.get(aa)
        if (0 < regi.shape[0] < 5):
            coin += 1
            x += 1
            y += 1
        else:
            no_coin += 1
            y += 1
        dic[aa] = (x, y) 

##### Calculamos los porcentajes de identificación por cada par posible de columnas

In [19]:
new_d = {}
for key, value in dic.iteritems():
    a, b = value
    if (a == 0 or b == 0):
        new_d[key] = 0
    else:
        new_d[key] = a * 1.0 / b

##### Ordenamos los procentajes descendentemente y los mostramos

In [20]:
import operator
sorted_x = sorted(new_d.items(), key=operator.itemgetter(1), reverse = True)

print sorted_x

[('lloc_cfgs', 1.0), ('nota_acces', 0.9933774834437086), ('pais_sistema_estranger', 0.875), ('lloc_secundaria', 0.8148148148148148), ('nacionalitat', 0.6666666666666666), ('universitat_procedencia', 0.6071428571428571), ('any_pau', 0.52), ('naixement', 0.5151515151515151), ('tipus_lloc_secundaria', 0.5), ('any_primera_matricula', 0), ('sexe', 0), ('sistema_educatiu_estranger', 0), ('priv_pub_secundaria', 0), ('tipus_lloc_cfgs', 0), ('becat', 0), ('priv_pub_cfgs', 0), ('id_via_acces', 0), ('id_enseny', 0), ('simultaneitat', 0)]


### Anonimización de los datos - Agregación

#### Visto que se pueden llegar a identificar alumnos conociendo alguno de sus datos, verificaremos que tecnicas de anonimización podemos aplciar para hacer imposible la identificación

#### Ejemplo 1. Redondear las notas de acceso a la universidad

In [21]:
notas_acceso = registers.nota_acces.values

##### Redondeamos las notas utilizando dos decimales

In [22]:
result = np.around(notas_acceso, 2)
total_ocurencias = 0
for nota in result:
    if (nota >= 5):
        ocurencia = sum(result == nota)
        if (ocurencia < 5):
            #print "Nota: " + str(nota) + " Ocurencia: " + str(ocurencia)
            total_ocurencias += 1
print "Total de ocurencias: " + str(total_ocurencias) + " de " + str(len(result)) + " posibles"

Total de ocurencias: 788 de 966 posibles


Podemos ver que el numero de ocurrencias es superior a 0, es decir, con cada uno de los 788 valores se ha podido identificar a menos de 5 alumnos.

##### Redondeamos las notas utilizando un decimal

In [23]:
result = np.around(notas_acceso, 1)
total_ocurencias = 0
for nota in result:
    if (nota >= 5):
        ocurencia = sum(result == nota)
        if (ocurencia < 5):
            #print "Nota: " + str(nota) + " Ocurencia: " + str(ocurencia)
            total_ocurencias += 1
print "Total de ocurencias: " + str(total_ocurencias) + " de " + str(len(result)) + " posibles"

Total de ocurencias: 51 de 966 posibles


Podemos ver que el numero de ocurrencias es superior a 0, es decir, con cada uno de los 51 valores se ha podido identificar a menos de 5 alumnos.

##### Redondeamos las notas sin utulizar decimales

In [24]:
result = np.around(notas_acceso, 0)
total_ocurencias = 0
for nota in result:
    if (nota >= 5):
        ocurencia = sum(result == nota)
        if (ocurencia < 5):
            print "Nota: " + str(nota) + " Ocurencia: " + str(ocurencia)
            total_ocurencias += 1
print "Total de ocurencias: " + str(total_ocurencias) + " de " + str(len(result)) + " posibles"

Total de ocurencias: 0 de 966 posibles


#####Podemos ver que el numero de ocurrencias es 0, es decir, para hacer imposible la identificación hace falta redondear las notas de acceso sin utilizar decimales

#### Ejemplo 1. Agregar de la edad

In [25]:
a_nacimiento = registers.naixement

##### Pasamos de años de nacimientos a edad

##### A partir del año de nacimiento, calculamos la edad de cada alumno

In [26]:
a_actual = datetime.datetime.now().year
edades = []
for a in a_nacimiento:
    edades.append(a_actual - a)

##### Agregamos la edad de modo que cada edad estará en una de las siguientes 3 categorías:
+ inferior a 25 años
+ superior a 25 años e inferior a 35 años
+ superior a 35 años

In [27]:
red_edades = []
for edad in edades:
    if edad < 25:
        red_edades.append("<25")
    elif edad > 35:
        red_edades.append(">35")
    else:
        red_edades.append(">25<35")

##### A continuación veremos el numero de alumnos por cada categoría

In [28]:
print "Numero de alumnos con edad inferior a 25 años: " + str(red_edades.count("<25"))
print "Numero de alumnos con edad superior a 25 años e inferior a 35 años: " + str(red_edades.count(">25<35"))
print "Numero de alumnos con edad superior a 35 años: " + str(red_edades.count(">35"))

Numero de alumnos con edad inferior a 25 años: 633
Numero de alumnos con edad superior a 25 años e inferior a 35 años: 308
Numero de alumnos con edad superior a 35 años: 25


##### Podemos ver que por cada categoría no se pueden identificar a menos de 5 alumnos que es el limite para considerar los datos anonimos.
##### Aunque se han elegido estas tres categorías, se podría intentar agregar los datos utilizando otras categorías.